# Intro to Using the NetHack Learning Dataset

There are two different sets of trajectories included in the NetHack Learning Dataset:
- **NLD-NAO**: state-only trajectories from 1.5 M human games played on nethack.alt.org
- **NLD-AA**: state-action-score trajectories from 100k NLE games played by the symbolic-bot winner of the 2021 NetHack Challenge

We also supply a small "taster" dataset, for quick iteration and playing around:
- **NLD-AA-Taster**: ~2,000 randomly chosen trajectories from **NLD-AA**

These trajectories can be used with the `TtyrecDataset` tool which allows for efficiently training on the datasets.  This tutorial describes how to create and use and visualize the dataset, using **NLD-AA-Taster**.



## Downloading the Data

For the time being data is available through various WeTransfer links in the DATASET.md file. Although generally this requires a browser to interface to download, it is also possible to use the command line (see here).

In this case, we use a publically available unzipped version of **NLD-AA-Taster** available on GitHub (or you can access the [zipped release]()).


## Install NLE

Make sure you have `nle` installed by following the instructions [in the repo README here](https://github.com/facebookresearch/nle). Either clone and install, or use pip. In this case, Colab struggles a bit to find cmake so we build from source:

## Setting up the Database

Adding datasets is easy - all you need is the path to the unzipped directory.

**NOTE** We call different functions to add trajectories generated by NLE (such as **NLD-AA**, **NLD-AA-Taster** or your own dataset) versus those generated from NAO (**NLD-NAO**).  

In [14]:
import nle.dataset as nld

In [15]:
# 1. Get the paths for your unzipped datasets
path_to_nld_aa_taster = "./data/nld-aa-taster/nle_data"

# 2. Chose a database name/path. By default, most methods with use nld.db.DB (='ttyrecs.db')
dbfilename = "ttyrecs.db"

if not nld.db.exists(dbfilename):
    # 3. Create the db and add the directory
    nld.db.create(dbfilename)
    nld.add_nledata_directory(path_to_nld_aa_taster, "taster-dataset", dbfilename)


# NB: To add the NLE-AA data, or any data generated from nle, use `add_nledata_directory`.
# nld.add_nledata_directory(path_to_nld_aa, "nld-aa", dbfilename)

# NB: To add the NLE-NAO data, use the `add_altorg_directory`.
# nld.add_altorg_directory(path_to_nld_nao, "nld-nao", dbfilename)


In [47]:
path_to_nld_aa_training = "./data/nld-aa/nle_data_train"
path_to_nld_aa_testing = "./data/nld-aa/nle_data_test"
nld.add_nledata_directory(path_to_nld_aa_training, "nld-aa-training", dbfilename)
nld.add_nledata_directory(path_to_nld_aa_testing, "nld-aa-testing", dbfilename)

Adding dataset 'nld-aa-training' ('./data/nld-aa/nle_data_train') to 'ttyrecs.db' 
Updated 'ttyrecs.db' in 0.70 sec. Size: 4.12 MB, Games: 11194
Adding dataset 'nld-aa-testing' ('./data/nld-aa/nle_data_test') to 'ttyrecs.db' 
Updated 'ttyrecs.db' in 0.70 sec. Size: 4.12 MB, Games: 11194
Adding dataset 'nld-aa-testing' ('./data/nld-aa/nle_data_test') to 'ttyrecs.db' 
Updated 'ttyrecs.db' in 0.17 sec. Size: 4.96 MB, Games: 2709
Updated 'ttyrecs.db' in 0.17 sec. Size: 4.96 MB, Games: 2709


In [49]:
path_to_nld_nao_training = "./data/nld-nao/nld_nao_train"
path_to_nld_nao_testing = "./data/nld-nao/nld_nao_test"
nld.add_altorg_directory(path_to_nld_nao_training, "nld-nao-training", dbfilename)
nld.add_altorg_directory(path_to_nld_nao_testing, "nld-nao-testing", dbfilename)

Adding dataset 'nld-nao-training' ('./data/nld-nao/nld_nao_train') to 'ttyrecs.db' 
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh363+:Zone.Identifier'
Found 1736841 games in './data/nld-nao/nld_nao_train/xlogfile.nh363+'
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh362:Zone.Identifier'
Found 1736841 games in './data/nld-nao/nld_nao_train/xlogfile.nh363+'
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh362:Zone.Identifier'
Found 167705 games in './data/nld-nao/nld_nao_train/xlogfile.nh362'
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh361dev:Zone.Identifier'
Found 167705 games in './data/nld-nao/nld_nao_train/xlogfile.nh362'
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh361dev:Zone.Identifier'
Found 20939 games in './data/nld-nao/nld_nao_train/xlogfile.nh361dev'
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh361:Zone.Identifier'
Found 20939 games in './data/nld-nao/nld_nao_train/xlogfile.nh361dev'
Found 0 games in '.

You can inspect the dataset using the database tooling:

In [16]:
# Create a connection to specify the database to use
db_conn = nld.db.connect(filename=dbfilename)

# Then you can inspect the number of games in each dataset:
print(f"NLD AA \"Taster\" Dataset has {nld.db.count_games('taster-dataset', conn=db_conn)} games.")

NLD AA "Taster" Dataset has 1934 games.


## Visualizing the Data

Next, to actually load the games for training you'll use the `TtyrecDataset` object:

In [72]:
dataset = nld.TtyrecDataset(
    "nld-aa-training",
    batch_size=1,
    seq_length=32,
    dbfilename=dbfilename,
    gameids=[12304]
)

This dataset above will return batches of 128 trajectories, returning sequential chunks of length 32.   That is, assuming the length of all trajectories is >>64, the first batch will give timesteps 0-31 of 128 games and the second batch will provide timesteps 32-63 for the same games, etc.

### Whats in the Observation?

In [74]:
minibatch = next(iter(dataset))
minibatch.keys()

dict_keys(['tty_chars', 'tty_colors', 'tty_cursor', 'timestamps', 'done', 'gameids', 'keypresses', 'scores'])

In [79]:
minibatch['tty_chars'][0][0].shape

(24, 80)

In [86]:
''.join([chr(a) for a in minibatch['tty_chars'][0][0][3]])

'                                                     -----                      '

The observation is made up of three components:
- `tty_chars` is a (batched) 2D np.array of the characters displayed at each point on the screen with shape: `[Batch, Time, H, W]`
- `tty_colors` is the associated colors for those characters
- `tty_cursor` provides the cursor position (NOTE: it's not always on the hero!)

These can be easily visualized usign the `tty_render` utility:

In [19]:
from nle.nethack import tty_render

In [98]:
batch_idx = 0
time_idx = 0
chars = minibatch['tty_chars'][batch_idx, time_idx]
colors = minibatch['tty_colors'][batch_idx, time_idx]
cursor = minibatch['tty_cursor'][batch_idx, time_idx]

print(tty_render(chars, colors, cursor))


Hello Agent, welcome to NetHack!  You are a neutral female gnomish              
                                                                  Archeologist.-
-More--                                                                         
                                                     -----                      
                                                     +...+                      
                                                     |[({|                      
                                                     .f..|                      
                                                     |@..-                      
                                                     +...|                      
                                                     -----                      
                                                                                
                                                                                
                           

### Extracting Inventory Information from Dataset

The dataset contains several ways to access inventory information:
1. **From TTY rendering** - parsing the visual inventory display
2. **From minibatch keys** - if available in the dataset
3. **By triggering inventory commands** - looking for 'i' keypresses

Let's explore all these methods:

In [39]:
# First, let's see what keys are available in our minibatch
print("=== Available Data in Minibatch ===")
print("Keys:", list(minibatch.keys()))
print()

# Check if inventory-related keys exist
inventory_keys = ['inv_glyphs', 'inv_letters', 'inv_oclasses', 'inv_strs']
available_inv_keys = [key for key in inventory_keys if key in minibatch.keys()]
print(f"Inventory-specific keys available: {available_inv_keys}")

# If no direct inventory keys, we'll need to parse from TTY or find inventory screens
if not available_inv_keys:
    print("No direct inventory keys found. We'll need to extract from TTY rendering or find inventory commands.")
else:
    print("Great! We have direct inventory data available.")

print(f"\nMinibatch shapes:")
for key, value in minibatch.items():
    if hasattr(value, 'shape'):
        print(f"  {key}: {value.shape}")
    else:
        print(f"  {key}: {type(value)}")

=== Available Data in Minibatch ===
Keys: ['tty_chars', 'tty_colors', 'tty_cursor', 'timestamps', 'done', 'gameids', 'keypresses', 'scores']

Inventory-specific keys available: []
No direct inventory keys found. We'll need to extract from TTY rendering or find inventory commands.

Minibatch shapes:
  tty_chars: (32, 32, 24, 80)
  tty_colors: (32, 32, 24, 80)
  tty_cursor: (32, 32, 2)
  timestamps: (32, 32)
  done: (32, 32)
  gameids: (32, 32)
  keypresses: (32, 32)
  scores: (32, 32)


In [40]:
# Method 1: Find when players opened inventory (pressed 'i')
print("=== Method 1: Finding Inventory Commands ===")

# Look for inventory keypresses (ASCII 105 = 'i')
inventory_keypress = ord('i')  # 105

# Search through the dataset for inventory commands
batch_with_inventory = None
timestep_with_inventory = None

for batch_idx in range(min(5, minibatch['keypresses'].shape[0])):  # Check first 5 batches
    for time_idx in range(minibatch['keypresses'].shape[1]):
        if minibatch['keypresses'][batch_idx, time_idx] == inventory_keypress:
            batch_with_inventory = batch_idx
            timestep_with_inventory = time_idx
            print(f"Found inventory command at batch {batch_idx}, timestep {time_idx}")
            break
    if batch_with_inventory is not None:
        break

if batch_with_inventory is None:
    print("No inventory commands found in current minibatch. Let's create a larger search...")
    
    # Create a larger dataset to search for inventory
    larger_dataset = nld.TtyrecDataset(
        "taster-dataset",
        batch_size=10,
        seq_length=100,
        dbfilename=dbfilename,
    )
    
    # Search multiple batches
    found_inventory = False
    for mb_idx, large_mb in enumerate(larger_dataset):
        if mb_idx > 3:  # Don't search forever
            break
            
        inventory_positions = (large_mb['keypresses'] == inventory_keypress)
        if inventory_positions.any():
            # Find the first occurrence
            batch_indices, time_indices = inventory_positions.nonzero()
            batch_with_inventory = batch_indices[0]
            timestep_with_inventory = time_indices[0]
            
            print(f"Found inventory command in batch {mb_idx}, game {batch_with_inventory}, timestep {timestep_with_inventory}")
            
            # Use this minibatch for analysis
            minibatch_with_inv = large_mb
            found_inventory = True
            break
    
    if not found_inventory:
        print("No inventory commands found. Using current minibatch for demonstration.")
        minibatch_with_inv = minibatch
        batch_with_inventory = 0
        timestep_with_inventory = 10  # Just pick a timestep
else:
    minibatch_with_inv = minibatch

print(f"Using batch {batch_with_inventory}, timestep {timestep_with_inventory} for analysis")

=== Method 1: Finding Inventory Commands ===
Found inventory command at batch 1, timestep 17
Using batch 1, timestep 17 for analysis


In [41]:
# Method 2: Analyze the inventory screen
print("=== Method 2: Analyzing Inventory Screen ===")

# Look at the screen right after the inventory command
if timestep_with_inventory + 1 < minibatch_with_inv['tty_chars'].shape[1]:
    next_timestep = timestep_with_inventory + 1
else:
    next_timestep = timestep_with_inventory

chars_before = minibatch_with_inv['tty_chars'][batch_with_inventory, timestep_with_inventory]
colors_before = minibatch_with_inv['tty_colors'][batch_with_inventory, timestep_with_inventory]
cursor_before = minibatch_with_inv['tty_cursor'][batch_with_inventory, timestep_with_inventory]

chars_after = minibatch_with_inv['tty_chars'][batch_with_inventory, next_timestep]
colors_after = minibatch_with_inv['tty_colors'][batch_with_inventory, next_timestep]
cursor_after = minibatch_with_inv['tty_cursor'][batch_with_inventory, next_timestep]

print("Screen BEFORE inventory command:")
print(tty_render(chars_before, colors_before, cursor_before))
print("\n" + "="*80 + "\n")

print("Screen AFTER inventory command:")
print(tty_render(chars_after, colors_after, cursor_after))

# Method 3: Extract inventory information from the text
print("\n=== Method 3: Parsing Inventory Information ===")

def extract_inventory_from_screen(chars):
    """Extract inventory items from TTY characters"""
    inventory_items = []
    
    # Convert chars to string representation
    screen_lines = []
    for row in range(chars.shape[0]):
        line = ''.join([chr(c) if 32 <= c <= 126 else ' ' for c in chars[row]])
        screen_lines.append(line.rstrip())
    
    # Look for common inventory patterns
    for i, line in enumerate(screen_lines):
        line = line.strip()
        
        # NetHack inventory lines typically start with a letter followed by ')'
        if len(line) > 2 and line[1] == ')' and line[0].isalpha():
            inventory_items.append({
                'slot': line[0],
                'description': line[2:].strip(),
                'line_number': i
            })
        
        # Also look for "You are carrying:" or similar inventory headers
        if 'carrying' in line.lower() or 'inventory' in line.lower():
            print(f"Inventory header found at line {i}: '{line}'")
    
    return inventory_items, screen_lines

# Extract from the inventory screen
inv_items, screen_lines = extract_inventory_from_screen(chars_after)

print(f"Found {len(inv_items)} inventory items:")
for item in inv_items:
    print(f"  {item['slot']}) {item['description']}")

if len(inv_items) == 0:
    print("No inventory items found in standard format.")
    print("Screen might show a different interface. Here are all non-empty lines:")
    for i, line in enumerate(screen_lines):
        if line.strip():
            print(f"  Line {i:2d}: '{line}'")

=== Method 2: Analyzing Inventory Screen ===
Screen BEFORE inventory command:

                                What do you want to name?                       
                                                                                
                                m - a monster                                   
                                i - a particular object in inventory            
                                o - the type of an object in inventory          
                                f - the type of an object upon the floor        
                                d - the type of an object on discoveries list   
                                a - record an annotation for the current level  
                                (end)                                           
                                                                                
                                                                                
                              

Then, the other elements of the batch are:
- `gameids`: The gameid for the game which the observation is from.
- `timestamps`: The time when the state was recorded, allowing you to understand how long the player took between frames.
- `keypresses`: The keypresses entered after seeing the observation at this timestep (which produces the observation at the next timestep).
- `scores`: The in-game score at this timestep (the result of the action at the previous timestep)
- `done`: Whether the gameid corresponding to the previous timestep's observation completed. If done is `True` this means that the observation at the current timestep is the beginning of the next gameid.

### Converting Actions from Keypresses to Environment Action Space

Note that the "actions" data is actually a keypress (eg ascii) entered not an action value corresponding to the actions in the nle environment.  To convert from keypresses to the action_space of the environment you can use an embedding as shown below:

In [35]:
import torch
from nle.env.tasks import NetHackChallenge


env = NetHackChallenge(
    savedir=None,  # Do not save any recordings. 
    character='@', # Randomly rotate through characters.
)

# Then use the environment actions to convert the keypresses.
embed_actions = torch.zeros((256, 1))
for i, a in enumerate(env.actions):
    embed_actions[a.value][0] = i
    
embed_actions = torch.nn.Embedding.from_pretrained(embed_actions)
keypresses = torch.Tensor(minibatch["keypresses"]).long()
actions = embed_actions(keypresses).squeeze(-1).long()

## Dataset Configuration Options
`shuffle`: While states within a trajectory are always returned sequentially, it is possible to turn on shuffling of the *gameids*.  When true, the order of the gameids sampled is shuffled but not the order of the `seq_length` chunks returned within a single gameid.

`loop_forever`: It is possible to have the iterator loop forever instead of cycling only through the dataset once.

`gameids`: You can specify a list of gameids to return instead of iterating through the full dataset.

`subselect_sql`: And, you can select even more complicated sets of games using specific sql queries.

**NB** A `gameid` of 0 indicates that that index is padded (with 0's).

**Example 1:** Lets create a small dataset of just 4 games, and see the shuffle functionality:

In [26]:
shuffle_small_dataset = nld.TtyrecDataset(
    "taster-dataset",
    batch_size=2,
    seq_length=6000,
    dbfilename=dbfilename,
    shuffle=True,
    loop_forever=False,
    gameids=[34,550,45],
)
for epoch in range(3):
    print(f"Epoch: {epoch}")
    for ind, mb in enumerate(shuffle_small_dataset):
        gameids = mb["gameids"][:, 0]
        print(f"  Batch {ind} first timestep gameids: {gameids}")
    print()


Epoch: 0
  Batch 0 first timestep gameids: [550  34]
  Batch 0 first timestep gameids: [550  34]
  Batch 1 first timestep gameids: [550  34]
  Batch 1 first timestep gameids: [550  34]
  Batch 2 first timestep gameids: [550  34]
  Batch 2 first timestep gameids: [550  34]
  Batch 3 first timestep gameids: [550  45]
  Batch 3 first timestep gameids: [550  45]
  Batch 4 first timestep gameids: [550  45]
  Batch 4 first timestep gameids: [550  45]
  Batch 5 first timestep gameids: [550  45]
  Batch 5 first timestep gameids: [550  45]
  Batch 6 first timestep gameids: [550  45]
  Batch 6 first timestep gameids: [550  45]
  Batch 7 first timestep gameids: [550  45]
  Batch 7 first timestep gameids: [550  45]
  Batch 8 first timestep gameids: [550  45]
  Batch 8 first timestep gameids: [550  45]
  Batch 9 first timestep gameids: [550  45]
  Batch 10 first timestep gameids: [550   0]

Epoch: 1
  Batch 9 first timestep gameids: [550  45]
  Batch 10 first timestep gameids: [550   0]

Epoch: 1
 

**Example 2:** We can train just on the data from a specific character, such as "mon-hum-neu-mal" by using the subselect_sql:

In [32]:
# Build the subselect sql query
subselect_sql = "SELECT gameid FROM games WHERE role=? AND race=?"
subselect_sql_args = ("Mon", "Hum")
batch_size = 10

# Build the dataset
monk_dataset = nld.TtyrecDataset(
    "taster-dataset",
    batch_size=batch_size,
    seq_length=2,
    dbfilename=dbfilename,
    subselect_sql=subselect_sql,
    subselect_sql_args=subselect_sql_args
)

# See from the error how there are fewer than 10k games despite the full dataset having 109k
print(f"Full Dataset has {nld.db.count_games('taster-dataset', conn=db_conn):,} games.")
print(f"Human Monk Subdataset Has: {len(monk_dataset._gameids)} games")

mb = next(iter(monk_dataset))

batch_idx = 0
time_idx = 0
chars = mb['tty_chars'][batch_idx, time_idx]
colors = mb['tty_colors'][batch_idx, time_idx]
cursor = mb['tty_cursor'][batch_idx, time_idx]

print(tty_render(chars, colors, cursor))

Full Dataset has 1,934 games.
Human Monk Subdataset Has: 142 games

Hello Agent, welcome to NetHack!  You are a neutral female human Monk.          
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                         

**Example 3**: Using a threadpool
You can also use a threadpool with the dataset which will speed it up considerably!

In [34]:
from concurrent.futures import ThreadPoolExecutor
import time


with ThreadPoolExecutor(max_workers=10) as tp:
    dataset = nld.TtyrecDataset(
        "taster-dataset",
        batch_size=100,
        seq_length=100,
        dbfilename=dbfilename,
        threadpool=tp
    )
    start = time.time()
    for i, mb in enumerate(dataset):
        if i == 10:
            break
    end = time.time()
    chars = mb['tty_chars'][batch_idx, time_idx]
    colors = mb['tty_colors'][batch_idx, time_idx]
    cursor = mb['tty_cursor'][batch_idx, time_idx]

    print(tty_render(chars, colors, cursor))
# NB this might be v slow on free Colab, try on laptop or server.
print(f"Loaded 100,000 frames in {end-start:.2f}s")


                                                                                
                                                                                
                              -----------                                       
                              |..Z......|              ----------------         
                              |..........#           ##...:...........|         
                              |.........|#           # |._............|         
                              ---------|-#           # |..............|         
        ----------------            #### #           # -.--------------         
        |.......{..<...-###    ######    ##          #  #####                   
        |..............|  ##   #          #        ###      #                   
        |...............@#d#####          #      ###        #######             
        ----------------    #-|-----      #-----##                #             
                           

**Example 4:** Getting Metadata

In [ ]:
dataset = nld.TtyrecDataset('taster-dataset', dbfilename=dbfilename)
mb = next(iter(dataset))
gameid = mb["gameids"][0][0]

chars = mb['tty_chars'][0, 0]
colors = mb['tty_colors'][0, 0]
cursor = mb['tty_cursor'][0, 0]

print(tty_render(chars, colors, cursor))

dict(dataset.get_meta(gameid))


Hello Agent, welcome to NetHack!  You are a neutral male gnomish Archeologist.  
                                                                                
                                                                                
                                                                                
                               --------                                         
                               |....$.|                                         
                               |..!{..|                                         
                               |......|                                         
                               |...@f(|                                         
                               |......|                                         
                               |.....)|                                         
                               --.-----                                         
                           

{'gameid': 816,
 'version': '3.6.6',
 'points': 7515,
 'deathdnum': 0,
 'deathlev': 5,
 'maxlvl': 5,
 'hp': 0,
 'maxhp': 49,
 'deaths': 1,
 'deathdate': 20220518,
 'birthdate': 20220518,
 'uid': 1185200751,
 'role': 'Arc',
 'race': 'Gno',
 'gender': 'Mal',
 'align': 'Neu',
 'name': 'Agent',
 'death': 'killed by a white unicorn',
 'conduct': '0xfc0',
 'turns': 22750,
 'achieve': '0x0',
 'realtime': 142,
 'starttime': 1652882603,
 'endtime': 1652882745,
 'gender0': 'Mal',
 'align0': 'Neu',
 'flags': '0x4'}

**Example 5** Generating and loading a custom dataset.

In [ ]:
import gym
import nle
import nle.dataset as nld
from datetime import datetime

def generate_rollouts(env):
    obs = env.reset()
    episodes = 0
    while episodes < 10:
        obs, reward, done, info = env.step(env.action_space.sample())
        if done:
            env.reset()
            episodes += 1

# 1. Create some envs, with a savedir directory 'path/to/save/X'
envA = gym.make("NetHackChallenge-v0", savedir="path/to/save/A", save_ttyrec_every=2)
envB = gym.make("NetHackScore-v0", character="Mon-Hum-Neu-Mal", savedir="path/to/save/B", save_ttyrec_every=1)

# 2. Generate rollouts
generate_rollouts(envA)
generate_rollouts(envB)

# 3. Add to directory, with given unique dataset name
name = f"dataset_{datetime.now().time()}"
if not nld.db.exists():
    nld.db.create()
nld.add_nledata_directory("path/to/save", name)

# 4. Use and enjoy!
dataset = nld.TtyrecDataset(name)
print(f"Dataset has {len(dataset._gameids)} entries!")



Adding dataset 'dataset_15:38:53.943302' ('path/to/save') to 'ttyrecs.db' 
Updated 'ttyrecs.db' in 0.00 sec. Size: 0.65 MB, Games: 15
Dataset has 15 entries!


**Example 6:** Use doctstrings - don't forget a lot of the classes and methods have docstrings. Have fun!

In [ ]:
help(nld.TtyrecDataset)

Help on class TtyrecDataset in module nle.dataset.dataset:

class TtyrecDataset(builtins.object)
 |  TtyrecDataset(dataset_name, batch_size=128, seq_length=32, rows=24, cols=80, dbfilename='ttyrecs.db', threadpool=None, gameids=None, shuffle=True, loop_forever=False, subselect_sql=None, subselect_sql_args=None)
 |  
 |  Dataset object to allow iteration through the ttyrecs found in our ttyrec
 |  database.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, dataset_name, batch_size=128, seq_length=32, rows=24, cols=80, dbfilename='ttyrecs.db', threadpool=None, gameids=None, shuffle=True, loop_forever=False, subselect_sql=None, subselect_sql_args=None)
 |      An iterable dataset to load minibatches of NetHack games from compressed
 |      ttyrec*.bz2 files into numpy arrays. (shape: [batch_size, seq_length, ...])
 |      
 |      This class makes use of a sqlite3 database at `dbfilename` to find the
 |      metadata and the location of files in a dataset. It then uses these to
 |   